In [48]:
! pip install langchain langchain-experimental openai python-dotenv ipython-sql tiktoken faiss-cpu

In [9]:
from getpass import getpass
import os

api_key = getpass('Enter your OpenAI API key:')
os.environ['OPENAI_API_KEY'] = api_key

Enter your OpenAI API key:··········


In [10]:
# Run basic query with OpenAI wrapper

from langchain.llms import OpenAI
llm = OpenAI(model_name="text-davinci-003")
llm("explain large language models in one sentence")

'\n\nLarge language models are models that use large amounts of text data to learn and generate text.'

In [11]:
# import schema for chat messages and ChatOpenAI in order to query chatmodels GPT-3.5-turbo or GPT-4

from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)
from langchain.chat_models import ChatOpenAI

In [12]:
chat = ChatOpenAI(model_name="gpt-3.5-turbo",temperature=0.3)
messages = [
    SystemMessage(content="You are an expert data scientist"),
    HumanMessage(content="Write a Python script that trains a neural network on simulated data ")
]
response=chat(messages)

print(response.content,end='\n')

Certainly! Here's an example of a Python script that trains a neural network on simulated data using the Keras library:

```python
import numpy as np
from keras.models import Sequential
from keras.layers import Dense

# Generate simulated data
np.random.seed(0)
X = np.random.rand(100, 2)
y = np.random.randint(2, size=100)

# Define the neural network model
model = Sequential()
model.add(Dense(4, input_dim=2, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X, y, epochs=10, batch_size=10)

# Evaluate the model
loss, accuracy = model.evaluate(X, y)
print(f'Loss: {loss}, Accuracy: {accuracy}')
```

In this script, we first generate simulated data using `numpy`. We then define a neural network model using `Sequential` from Keras and add two dense layers. The first layer has 4 neurons and uses the ReLU activation function, while the second layer ha

In [13]:
# Import prompt and define PromptTemplate

from langchain import PromptTemplate

template = """
You are an expert data scientist with an expertise in building deep learning models.
Explain the concept of {concept} in a couple of lines
"""

prompt = PromptTemplate(
    input_variables=["concept"],
    template=template,
)

In [14]:
# Run LLM with PromptTemplate

llm(prompt.format(concept="autoencoder"))

'\nAutoencoders are machine learning algorithms that can learn a compressed representation of input data, typically for the purpose of dimensionality reduction or feature extraction. They are composed of an encoder, which compresses the input data into a lower dimensional representation, and a decoder, which reconstructs the original input from the lower dimensional representation.'

In [15]:
# Import LLMChain and define chain with language model and prompt as arguments.

from langchain.chains import LLMChain
chain = LLMChain(llm=llm, prompt=prompt)

# Run the chain only specifying the input variable.
print(chain.run("autoencoder"))


Autoencoders are a type of neural network that are trained to encode and decode high dimensional data into a lower dimensional representation. They learn to reconstruct the input data from the latent representation, allowing for data compression and feature extraction.


In [16]:
# Define a second prompt

second_prompt = PromptTemplate(
    input_variables=["ml_concept"],
    template="Turn the concept description of {ml_concept} and explain it to me like I'm five in 500 words",
)
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [17]:
# Define a sequential chain using the two chains above: the second chain takes the output of the first chain as input

from langchain.chains import SimpleSequentialChain
overall_chain = SimpleSequentialChain(chains=[chain, chain_two], verbose=True)

# Run the chain specifying only the input variable for the first chain.
explanation = overall_chain.run("autoencoder")
print(explanation)



> Entering new SimpleSequentialChain chain...

An autoencoder is a type of artificial neural network used to learn efficient data encodings in an unsupervised manner. It consists of an encoder, which maps the input data to a hidden representation, and a decoder, which reconstructs the input data from the hidden representation.


An autoencoder is a special type of machine learning tool used to make information easier to store and access. It can take data like numbers, words, or images, and then “encode” it into a simpler format that takes up less space. Think of it like a translator, changing something from one language to another. 

The autoencoder is made up of two parts. The first part is called the “encoder” and it takes the data and turns it into a simpler form. It is like a box with a lock on it. The data goes in one end, and the encoder puts it into the box and locks it up. 

The second part of the autoencoder is called the “decoder”. This part of the autoencoder takes the dat

In [18]:
# Import utility for splitting up texts and split up the explanation given above into document chunks

from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 100,
    chunk_overlap  = 0,
)

texts = text_splitter.create_documents([explanation])


In [19]:
# Individual text chunks can be accessed with "page_content"

texts[0].page_content

'An autoencoder is a special type of machine learning tool used to make information easier to store'

In [26]:
# Import and instantiate OpenAI embeddings

from langchain.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

In [27]:
# Turn the first text chunk into a vector with the embedding

query_result = embeddings.embed_query(texts[0].page_content)
print(query_result)

[-0.03385571295143853, 0.0008082212652876269, 0.02040123137061156, -0.024700983663872824, 0.0038090905965643697, 0.00305211465601134, 0.005406971868749722, -0.017586377885897293, -0.003495970409731291, -0.04320412752675291, 0.015068504535421443, 0.04741349310404544, 0.01257645489656803, 0.0006524681628592886, 0.005594198434625136, 0.010936609161673667, 0.012686208464931034, -0.004357857972718118, 0.0005729776296687171, -0.0038704233086514815, -0.03111833432688177, 0.01932952126124905, -0.014125915778138765, -0.031066685040991797, -0.02299657771859982, -0.005481216902309327, 0.015145977532933851, -0.03778101397559406, 0.00449666368231764, -0.011517656643016738, 0.02665072232013937, -0.026056762051662533, -0.010258719967778813, -0.049815157889575225, -0.03401065894646335, -0.012240738263573407, 0.008618874232884452, -0.022028164318372154, 0.023461415703674274, 0.006123597561400782, 0.025023789372378775, 0.015688288515520718, -0.004189999656220807, -0.02548862735745323, -0.005539321650443

In [42]:
# Import and initialize FAISS
import os
from langchain.vectorstores import FAISS

In [43]:
db = FAISS.from_documents(texts, embeddings)

In [44]:
# Do a simple vector similarity search

query = "What is magical about an autoencoder?"
result = db.similarity_search(query)

print(result)

[Document(page_content='An autoencoder is a special type of machine learning tool used to make information easier to store'), Document(page_content='The autoencoder works in an “unsupervised” way, which means it does not need someone to tell it'), Document(page_content='The autoencoder is made up of two parts. The first part is called the “encoder” and it takes the'), Document(page_content='The second part of the autoencoder is called the “decoder”. This part of the autoencoder takes the')]


In [46]:
# Import Python REPL tool and instantiate Python agent

from langchain_experimental.agents.agent_toolkits import create_python_agent
from langchain_experimental.tools import PythonREPLTool
from langchain.python import PythonREPL
from langchain.llms.openai import OpenAI

agent_executor = create_python_agent(
    llm=OpenAI(temperature=0, max_tokens=1000),
    tool=PythonREPLTool(),
    verbose=True
)

In [47]:
# Execute the Python agent

agent_executor.run("Find the roots (zeros) if the quadratic function 3 * x**2 + 2*x -1")



> Entering new AgentExecutor chain...


 I need to solve a quadratic equation
Action: Python_REPL
Action Input: from scipy.optimize import root
Observation: 
Thought: I need to define the function
Action: Python_REPL
Action Input: def f(x): return 3 * x**2 + 2*x -1
Observation: 
Thought: I need to find the roots of the function
Action: Python_REPL
Action Input: root(f, [0, 1])
Observation: 
Thought: I now know the final answer
Final Answer: The roots of the quadratic function 3 * x**2 + 2*x -1 are -1.0 and 0.3333333333333333.

> Finished chain.


'The roots of the quadratic function 3 * x**2 + 2*x -1 are -1.0 and 0.3333333333333333.'